In [1]:
import networkx as nx
import numpy as np
import pandas as pd
from scipy import sparse
from scipy.sparse import csr_matrix
from scipy.sparse import coo_matrix
from time import perf_counter

In [2]:
filename = './datasets/facebook_combined.txt'
# filename = './datasets/amazon0302.mtx'

In [3]:
t0 = perf_counter()
testgraph = nx.read_edgelist(filename, comments='#', delimiter=None, create_using=nx.Graph, nodetype=None, data=True, edgetype=None, encoding='utf-8')
t_load = perf_counter() - t0

In [4]:
## define the partition index
adj_matrix_dim = len(testgraph.nodes)
partition_num = 16 ## can be set a variable
partition_index = np.zeros(partition_num + 1, dtype=np.int32)
for i in range(partition_num):
    partition_index[i+1] = np.int32(adj_matrix_dim*(i+1)/partition_num)
print(partition_index)

[   0  252  504  757 1009 1262 1514 1767 2019 2271 2524 2776 3029 3281
 3534 3786 4039]


In [5]:
from multiprocessing.pool import ThreadPool as Pool

pool_size = 16

def worker(i, partition_index, G_part, testgraph):
    for edge_list in testgraph.edges:
        if((int(edge_list[1]) >= partition_index[i]) and (int(edge_list[1]) < partition_index[i+1])):
            G_part[i].add_edge(edge_list[0], edge_list[1])

pool = Pool(pool_size)

In [6]:
G_part = []
for i in range(partition_num):
    G_part.append(nx.DiGraph())

In [7]:
for i in range(partition_num):
    pool.apply_async(worker, (i, partition_index, G_part, testgraph, ))

In [8]:
for i in range(partition_num):
    print("graph partition ", i, ": ", G_part[i])

graph partition  0 :  DiGraph with 252 nodes and 1431 edges
graph partition  1 :  DiGraph with 471 nodes and 3219 edges
graph partition  2 :  DiGraph with 401 nodes and 3255 edges
graph partition  3 :  DiGraph with 347 nodes and 2036 edges
graph partition  4 :  DiGraph with 388 nodes and 2980 edges
graph partition  5 :  DiGraph with 618 nodes and 6036 edges
graph partition  6 :  DiGraph with 856 nodes and 10612 edges
graph partition  7 :  DiGraph with 1216 nodes and 9408 edges
graph partition  8 :  DiGraph with 448 nodes and 7399 edges
graph partition  9 :  DiGraph with 634 nodes and 12308 edges
graph partition  10 :  DiGraph with 893 nodes and 9937 edges
graph partition  11 :  DiGraph with 417 nodes and 3793 edges
graph partition  12 :  DiGraph with 620 nodes and 5722 edges
graph partition  13 :  DiGraph with 852 nodes and 4958 edges
graph partition  14 :  DiGraph with 360 nodes and 2178 edges
graph partition  15 :  DiGraph with 576 nodes and 2962 edges


In [9]:
## Get partition graph, maybe need to be parallel
# G_part = []
# for i in range(partition_num):
#     G_part.append(nx.DiGraph())

for edge_list in testgraph.edges:
    for i in range(partition_num):
        if ((int(edge_list[1]) >= partition_index[i]) and (int(edge_list[1]) < partition_index[i+1])):
            G_part[i].add_edge(edge_list[0], edge_list[1])


In [10]:
for i in range(partition_num):
    print("graph partition ", i, ": ", G_part[i])

graph partition  0 :  DiGraph with 252 nodes and 1431 edges
graph partition  1 :  DiGraph with 471 nodes and 3219 edges
graph partition  2 :  DiGraph with 401 nodes and 3255 edges
graph partition  3 :  DiGraph with 347 nodes and 2036 edges
graph partition  4 :  DiGraph with 388 nodes and 2980 edges
graph partition  5 :  DiGraph with 618 nodes and 6036 edges
graph partition  6 :  DiGraph with 856 nodes and 10612 edges
graph partition  7 :  DiGraph with 1216 nodes and 9408 edges
graph partition  8 :  DiGraph with 448 nodes and 7399 edges
graph partition  9 :  DiGraph with 634 nodes and 12308 edges
graph partition  10 :  DiGraph with 893 nodes and 9937 edges
graph partition  11 :  DiGraph with 417 nodes and 3793 edges
graph partition  12 :  DiGraph with 620 nodes and 5722 edges
graph partition  13 :  DiGraph with 852 nodes and 4958 edges
graph partition  14 :  DiGraph with 360 nodes and 2178 edges
graph partition  15 :  DiGraph with 576 nodes and 2962 edges


In [11]:
## triangle count in sub-graphs (need be parallel)
triangle_count = 0
node_triangle = np.zeros(len(testgraph.nodes))
for g_index in range(partition_num): ## could be executed parallelly in different machines
    for edge_list in testgraph.edges: ## traverse all the edges in the original graph
        if ((G_part[g_index].has_node(edge_list[0])) and (G_part[g_index].has_node(edge_list[1]))):
            srcSet = G_part[g_index].adj[edge_list[0]]
            dstSet = G_part[g_index].adj[edge_list[1]]
            for node_obj in srcSet:
                if node_obj in dstSet:
                    triangle_count += 1
                    node_triangle[int(edge_list[0])] += 1
                    node_triangle[int(edge_list[1])] += 1
                    node_triangle[int(node_obj)] += 1

print (triangle_count)
## print (node_triangle)


1612010


In [12]:
testgraph_golden = nx.read_edgelist(filename, comments='#', delimiter=None, create_using=None, nodetype=None, data=True, edgetype=None, encoding='utf-8')
triangle_golden = nx.triangles(testgraph_golden)
result_golden = int(sum(triangle_golden.values())/3)
print (result_golden)
## print (triangle_golden)

1612010
